# Lecture: Coping with missing data in python

At first, we are looking at a small exemplary dataset with missing values.

In [ ]:
import pandas as pd

df = pd.read_csv('../../data/propertydata.csv')

print(df)

Since there are some inconsistent encodings of NaN values we first preprocess the data to a more consistent format by standardization.

In [ ]:
df = pd.read_csv('../../data/propertydata.csv', na_values=['na', '--'])
print(df)

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt

msno.matrix(df)
plt.plot()

A very simple listwise deletion method would lead to an unsatisfactory result.

In [ ]:
# Simulate listwise deletion
print(df.dropna())

### The feature ST_NUM

At first, we look at the feature `ST_NUM`, the house number of the property. Obviously, it is a discrete numerical value.

In [ ]:
print(df[['ST_NUM']])
df[['ST_NUM']].describe()

 Imputing with `mean_strategy` is questionable due to the context and requires additional rounding at the end.

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
df['ST_NUM']=imp_mean.fit_transform(df[['ST_NUM']])
df['ST_NUM']= df['ST_NUM'].astype(int)
print(df['ST_NUM'])

Another possibility would be a random choice between the min and max values of the range of `ST_NUM`. This can be implemented with the `random` package and the `randint(min,max)` method.

### The feature OWN_OCCUPIED
Obviously, this is a binary categorial feature. There is not a missing entry only but also an invalid entry (number 12 in line 3).

In [ ]:
print(df)

To get a consistent feature, we need to go through all the entries and get rid of false datatypes.

In [ ]:
# Handle OWN_OCCUPIED
cnt = 0
for row in df['OWN_OCCUPIED']:
    try:
       # Try to cast value to int
        int(row)
        # If possible, replace that value
        df.loc[cnt, 'OWN_OCCUPIED'] = np.nan
    except ValueError:
        pass
    cnt += 1
print(df['OWN_OCCUPIED'])

In [ ]:
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[['OWN_OCCUPIED']]=imp_mean.fit_transform(df[['OWN_OCCUPIED']])
print(df['OWN_OCCUPIED'])

### The feature NUM_BATH

The same holds for this feature. We need to go through all entries and get rid of inconsistent datatypes.

In [ ]:
# Handle NUM_BATH
cnt = 0
for row in df['NUM_BATH']:
    try:
        # Try to cast value to float
        int(row) #float(row)
    except ValueError:
        # If NOT possible, replace that value
        df.loc[cnt, 'NUM_BATH'] = np.nan
    cnt += 1
print(df['NUM_BATH'])

In [ ]:
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
df[['NUM_BATH']]=imp_mean.fit_transform(df[['NUM_BATH']])
df['NUM_BATH']= df['NUM_BATH'].astype(int)
print(df['NUM_BATH'])

### The feature NUM_BEDROOMS
The feature NUM_BEDROOMS is a discrete numerical feature.

In [ ]:
print(df[['NUM_BEDROOMS']])

It is reasonable to either impute the rounded mean value or to use a more profound method like imputing integer values within the range according to a probability distribution of the entries. The former can be done analogously to the imputation methods shown above.

In [ ]:
print(df[['NUM_BEDROOMS']])

The latter can be implemented in two steps:
1. compute the discrete probability distribution of the used values
2. impute according to this distribution

In [ ]:
# For step 1 count the number of appearing values
counts = df['NUM_BEDROOMS'].value_counts()

# and divide them by the number of entries
probabilities = counts / counts.sum()

print(probabilities)


In [ ]:
for idx, row in enumerate(df['NUM_BEDROOMS']):
    if pd.isnull(row):
        df.loc[idx, 'NUM_BEDROOMS'] = np.random.choice(probabilities.index, p = probabilities)

print(df['NUM_BEDROOMS'])

### The feature SQ_FT

This is a numerical feature which can be easily imputed with mean values in the given context.

In [ ]:
# Perform mean imputation and down-cast to int to get rid of values like 2.167 bedrooms
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

df['SQ_FT'] = imp_mean.fit_transform(df[['SQ_FT']])
df['SQ_FT'] = df['SQ_FT'].astype(int)

In [ ]:
print(df)

In [ ]:
msno.matrix(df)
plt.plot()